In [79]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import dataframe_image as dfi

import warnings
warnings.simplefilter('ignore')

In [80]:
df = pd.read_excel('project_ledger.xlsx', sheet_name = 'ERCIP Obligation Ledger', header = 3, usecols=lambda x: 'Unnamed' not in x)
df_vis = df[['FY of Funding Action','Project #', 'P&D Funding Actions','Construction Funding Actions']]
df_vis.head(25)

,FY of Funding Action,Project #,P&D Funding Actions,Construction Funding Actions
0,2017,P-1337,139000.00,NaN
1,2017,P-1337,NaN,1395000.00
2,2017,P-1337,NaN,-1395000.00
3,2017,P-480,-124000.00,NaN
4,2017,P-143,-15000.00,NaN
5,2018,P-975,1001000.00,NaN
6,2018,P-938,NaN,6185000.00
7,2018,P-1487,NaN,9750000.00
8,2018,P-266,NaN,-589613.00
9,2018,P-1487,NaN,-13664039.00


In [81]:
#create functions to calculate values
def pos(col): 
  return col[col > 0].sum()
  
def neg(col): 
  return col[col < 0].sum()

df_grouped = df_vis.groupby(by = 'FY of Funding Action')

#apply lambda function to sum positive and negative values separately
pd_total = df_grouped['P&D Funding Actions'].agg([('spending', neg),
                          ('funding', pos)
                          ])
c_total = df_grouped['Construction Funding Actions'].agg([('spending', neg),
                          ('funding', pos)
                          ])
pd_total['spending'] = pd_total['spending'] * -1
c_total['spending'] = c_total['spending'] * -1

pd_total['balance'] = pd_total['funding'] - pd_total['spending'] 
c_total['balance'] = c_total['funding'] - c_total['spending'] 

pd_total.reset_index(inplace = True)
c_total.reset_index(inplace = True)

In [82]:
#rename columns
c_total.columns = ['FY', 'Spending', 'Funding', 'Balance (Const)']
pd_total.columns = ['FY', 'Spending', 'Funding', 'Balance (P&D)']

#type to int
pd_total = pd_total.astype({"Funding":'int', "Spending":'int', "Balance (P&D)": 'int'}) 
c_total = c_total.astype({"Funding":'int', "Spending":'int', "Balance (Const)": 'int'})

#set FY to index
pd_total.set_index('FY', inplace = True)
c_total.set_index('FY', inplace = True)

#transpose
pd_total = pd_total.T
c_total = c_total.T

In [83]:
#pd_total.set_index('FY', inplace = True)
pd_total.style.set_caption("P&D Balance Sheet")

FY,2017,2018,2019,2020,2021,2022,2023
Spending,139000,1001000,3957000,1165000,374395,805867,0
Funding,139000,1001000,6767862,1165000,405000,5759000,4377000
Balance (P&D),0,0,2810862,0,30605,4953132,4377000


In [84]:
dfi.export(pd_total, 'website/static/images/pd_5.png', table_conversion = 'matplotlib')

In [85]:

c_total.style.set_caption("Construction Balance Sheet")
dfi.export(c_total, 'website/static/images/construction_5.png', table_conversion = 'matplotlib')